In [1]:
import os
import zipfile
import pandas as pd

In [2]:
# Descargar el dataset de "Flight Status Prediction" desde Kaggle y extraerlo
!kaggle datasets download robikscube/flight-delay-dataset-20182022

Dataset URL: https://www.kaggle.com/datasets/robikscube/flight-delay-dataset-20182022
License(s): CC0-1.0
100% 3.73G/3.73G [02:55<00:00, 31.5MB/s]
100% 3.73G/3.73G [02:55<00:00, 22.8MB/s]


In [3]:
# Extraer el archivo zip descargado
with zipfile.ZipFile("/content/flight-delay-dataset-20182022.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/flight_data")


# Ruta donde están los archivos extraídos
folder_path = "/content/flight_data/"

In [4]:
# Teniendo en cuenta el tamaño de las bases de datos, para el preprocesamiento solo usaremos una muestra, que será los datos para 2018
# Lista de los archivos CSV que quieres leer
file_names = [
    "Combined_Flights_2018.csv"
]

# Diccionario para almacenar los DataFrames
dataframes = {}

# Leer cada archivo CSV y almacenarlo en el diccionario
for file in file_names:
    file_path = os.path.join(folder_path, file)
    year = file.replace("Combined_Flights_", "").replace(".csv", "")  # Extraer el año de registros
    dataframes[year] = pd.read_csv(file_path)

In [5]:
# Ejemplo: mostrar las primeras filas del DataFrame
print(dataframes['2018'].head())

   FlightDate            Airline Origin Dest  Cancelled  Diverted  CRSDepTime  \
0  2018-01-23  Endeavor Air Inc.    ABY  ATL      False     False        1202   
1  2018-01-24  Endeavor Air Inc.    ABY  ATL      False     False        1202   
2  2018-01-25  Endeavor Air Inc.    ABY  ATL      False     False        1202   
3  2018-01-26  Endeavor Air Inc.    ABY  ATL      False     False        1202   
4  2018-01-27  Endeavor Air Inc.    ABY  ATL      False     False        1400   

   DepTime  DepDelayMinutes  DepDelay  ...  WheelsOff  WheelsOn  TaxiIn  \
0   1157.0              0.0      -5.0  ...     1211.0    1249.0     7.0   
1   1157.0              0.0      -5.0  ...     1210.0    1246.0    12.0   
2   1153.0              0.0      -9.0  ...     1211.0    1251.0    11.0   
3   1150.0              0.0     -12.0  ...     1207.0    1242.0    11.0   
4   1355.0              0.0      -5.0  ...     1412.0    1448.0    11.0   

   CRSArrTime  ArrDelay  ArrDel15  ArrivalDelayGroups  ArrTime

In [6]:
data = dataframes['2018']

In [7]:
# veamos informacion de la base de datos
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5689512 entries, 0 to 5689511
Data columns (total 61 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   FlightDate                               object 
 1   Airline                                  object 
 2   Origin                                   object 
 3   Dest                                     object 
 4   Cancelled                                bool   
 5   Diverted                                 bool   
 6   CRSDepTime                               int64  
 7   DepTime                                  float64
 8   DepDelayMinutes                          float64
 9   DepDelay                                 float64
 10  ArrTime                                  float64
 11  ArrDelayMinutes                          float64
 12  AirTime                                  float64
 13  CRSElapsedTime                           float64
 14  ActualElapsedTime 

In [8]:
# vamos a eliminar a los vuelos que fueron desviados, ya que como tal no nos interesan para nuestro analisis

data = data[data["Diverted"] != 1]

In [9]:
# Teniendo en cuenta el número de columnas, vamos a eliminar algunas de las que consideramos menos relevantes para nuestro analisis:

eliminar = [
    'Quarter', # el mes ya captura la informacion de esta variable
    'Marketing_Airline_Network',
    'IATA_Code_Marketing_Airline', # el codigo no es unico, ademas ya existe otra variable que identifica unicamente los aviones
    'IATA_Code_Originally_Scheduled_Code_Share_Airline', # mismo que la variable anterior
    'IATA_Code_Operating_Airline', # mismo que la variable anterior
    'Duplicate',  # Ejemplo: columna de duplicados
    'Diverted',  # ex-post
    'DivAirportLandings',
    'CancellationCode',  # Es una variable ex-post, ademas que no estamos analizando razones de cancelamiento
    'OriginStateName', # redudante con OriginState
    'DestinyStateName', # redudante con OriginState
]
data = data.drop(columns=[col for col in data.columns if any(c in col for c in eliminar)])

In [10]:
# veamos los datos nulos

# Visualizar valores nulos
print(data.isnull().sum())

FlightDate                                     0
Airline                                        0
Origin                                         0
Dest                                           0
Cancelled                                      0
CRSDepTime                                     0
DepTime                                    85380
DepDelayMinutes                            86565
DepDelay                                   86565
ArrTime                                    88383
ArrDelayMinutes                            88938
AirTime                                    95316
CRSElapsedTime                                 3
ActualElapsedTime                          88392
Distance                                       0
Year                                           0
Month                                          0
DayofMonth                                     0
DayOfWeek                                      0
Operated_or_Branded_Code_Share_Partners        0
DOT_ID_Marketing_Air

In [11]:
# vemos que tenemos bastantes variables con más de 80 mil valores nulos (esto es alrededor del 2% del total de nuestras observaciones)
# algunas de estas variables son cruciales para nuestro analisis, ya que se relacionan directamente con el retraso o no de un vuelo

# por ello, en primera instancia, vamos a eliminar todas aquellas observaciones con valores nulos en las siguientes variables:

data = data.dropna(subset=['DepTime', 'DepDelayMinutes', 'ArrTime', 'DepDelay', 'ArrDelayMinutes', 'AirTime', 'ActualElapsedTime', 'DepDel15',
                           'DepartureDelayGroups', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'ArrDelay', 'ArrDel15', 'ArrivalDelayGroups'])

In [12]:
# Visualizar valores nulos
print(data.isnull().sum()) ## vemos que ya eliminamos todos los valores nulos

FlightDate                                 0
Airline                                    0
Origin                                     0
Dest                                       0
Cancelled                                  0
CRSDepTime                                 0
DepTime                                    0
DepDelayMinutes                            0
DepDelay                                   0
ArrTime                                    0
ArrDelayMinutes                            0
AirTime                                    0
CRSElapsedTime                             0
ActualElapsedTime                          0
Distance                                   0
Year                                       0
Month                                      0
DayofMonth                                 0
DayOfWeek                                  0
Operated_or_Branded_Code_Share_Partners    0
DOT_ID_Marketing_Airline                   0
Flight_Number_Marketing_Airline            0
Operating_

In [13]:
# veamos el numero de observaciones
print(data.shape[0]) ## vemos que no perdimos casi observaciones

5578618


In [14]:
# transformamos la variable de fecha a formato date

data['FlightDate'] = pd.to_datetime(data['FlightDate'])

In [15]:
data.head()

,FlightDate,Airline,Origin,Dest,Cancelled,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,...,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup
0,2018-01-23,Endeavor Air Inc.,ABY,ATL,False,1202,1157.0,0.0,-5.0,1256.0,...,14.0,1211.0,1249.0,7.0,1304,-8.0,0.0,-1.0,1300-1359,1
1,2018-01-24,Endeavor Air Inc.,ABY,ATL,False,1202,1157.0,0.0,-5.0,1258.0,...,13.0,1210.0,1246.0,12.0,1304,-6.0,0.0,-1.0,1300-1359,1
2,2018-01-25,Endeavor Air Inc.,ABY,ATL,False,1202,1153.0,0.0,-9.0,1302.0,...,18.0,1211.0,1251.0,11.0,1304,-2.0,0.0,-1.0,1300-1359,1
3,2018-01-26,Endeavor Air Inc.,ABY,ATL,False,1202,1150.0,0.0,-12.0,1253.0,...,17.0,1207.0,1242.0,11.0,1304,-11.0,0.0,-1.0,1300-1359,1
4,2018-01-27,Endeavor Air Inc.,ABY,ATL,False,1400,1355.0,0.0,-5.0,1459.0,...,17.0,1412.0,1448.0,11.0,1500,-1.0,0.0,-1.0,1500-1559,1


In [16]:
# algunas de las variables de tiempo (de llegada y de salida) estan en formato float, por lo que las pasaremos a int

data['DepTime'] = data['DepTime'].astype(int)
data['ArrTime'] = data['ArrTime'].astype(int)

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5578618 entries, 0 to 5689511
Data columns (total 54 columns):
 #   Column                                   Dtype         
---  ------                                   -----         
 0   FlightDate                               datetime64[ns]
 1   Airline                                  object        
 2   Origin                                   object        
 3   Dest                                     object        
 4   Cancelled                                bool          
 5   CRSDepTime                               int64         
 6   DepTime                                  int64         
 7   DepDelayMinutes                          float64       
 8   DepDelay                                 float64       
 9   ArrTime                                  int64         
 10  ArrDelayMinutes                          float64       
 11  AirTime                                  float64       
 12  CRSElapsedTime                   

In [18]:
# generemos una variable dummy que indica si el vuelo tuvo retraso (en su llegado) o no

data['Arr_Delayed'] = data['ArrDelay'] > 15

In [20]:
# vamos a estandarizar las variables categoricas

data['Airline'] = data['Airline'].astype('category').cat.codes
data['Origin'] = data['Origin'].astype('category').cat.codes
data['Dest'] = data['Dest'].astype('category').cat.codes
data['Operated_or_Branded_Code_Share_Partners'] = data['Operated_or_Branded_Code_Share_Partners'].astype('category').cat.codes
data['Operating_Airline'] = data['Operating_Airline'].astype('category').cat.codes
data['Tail_Number'] = data['Tail_Number'].astype('category').cat.codes
data['OriginCityName'] = data['OriginCityName'].astype('category').cat.codes
data['OriginState'] = data['OriginState'].astype('category').cat.codes
data['DestCityName'] = data['DestCityName'].astype('category').cat.codes
data['DestState'] = data['DestState'].astype('category').cat.codes
data['DepTimeBlk'] = data['DepTimeBlk'].astype('category').cat.codes
data['ArrTimeBlk'] = data['ArrTimeBlk'].astype('category').cat.codes

In [21]:
data.head() ## de esta forma ya hemos terminado el procesamiento de los datos

,FlightDate,Airline,Origin,Dest,Cancelled,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,Arr_Delayed
0,2018-01-23,11,4,22,False,1202,1157,0.0,-5.0,1256,...,1211.0,1249.0,7.0,1304,-8.0,0.0,-1.0,8,1,False
1,2018-01-24,11,4,22,False,1202,1157,0.0,-5.0,1258,...,1210.0,1246.0,12.0,1304,-6.0,0.0,-1.0,8,1,False
2,2018-01-25,11,4,22,False,1202,1153,0.0,-9.0,1302,...,1211.0,1251.0,11.0,1304,-2.0,0.0,-1.0,8,1,False
3,2018-01-26,11,4,22,False,1202,1150,0.0,-12.0,1253,...,1207.0,1242.0,11.0,1304,-11.0,0.0,-1.0,8,1,False
4,2018-01-27,11,4,22,False,1400,1355,0.0,-5.0,1459,...,1412.0,1448.0,11.0,1500,-1.0,0.0,-1.0,10,1,False
